Importamos librerias necesarias

In [1]:
import a_funciones as fn ### importar modulo de  funciones
from a_funciones import * ### importar toddas llas funciones del archivo
import pandas as pd ### para manejo de datos
import sqlite3 as sql
import joblib
#!pip install openpyxl
import openpyxl
import numpy as np

import warnings
from sklearn.preprocessing import StandardScaler

# Suprimir advertencias específicas de sklearn
warnings.filterwarnings("ignore", category=UserWarning)

El despliegue consiste en dejar todo el código listo para una ejecucion automática en el periodo definido:
En este caso se ejecutara el proceso de entrenamiento y prediccion anualmente.

# Conectarse a base de datos preprocesada

In [2]:
con=sql.connect('bases de datos RRHH/datos.db')
cur=con.cursor()

#Crear dataframe para el modelado(2016)

In [3]:
df = pd.read_sql("""select * from completo
where InfoDate='2016-12-31'
""",con=con)

# Preparación de los datos nuevos

In [4]:
df_t= fn.preparar_datos(df)

# Cargar modelo entrenado

In [5]:
rf_final = joblib.load("salidas\\rf_final.pkl")

# Realizar predicciones

In [6]:
predicciones = rf_final.predict(df_t)
pd_pred = pd.DataFrame(predicciones, columns=['Atrition'])

# Crear DataFrame con predicciones
perf_pred = pd.concat([df['EmployeeID'], df_t, pd_pred], axis=1)

# Guardar predicciones en archivos
perf_pred[['EmployeeID', 'Atrition']].to_excel("salidas\\prediccion.xlsx")
#Guardar importancia de las caracteristicas a la hora de predecir
feature_names = df_t.columns


In [7]:
importances = pd.DataFrame({'Feature': feature_names, 'Importance': rf_final.feature_importances_})
importances.to_excel("salidas\\importances.xlsx")

In [8]:
# Ver las 15 predicciones más bajas
emp_pred_bajo = perf_pred.sort_values(by=["Atrition"], ascending=False).head(15)
emp_pred_bajo.set_index('EmployeeID', inplace=True)
pred = emp_pred_bajo.T
print(pred)

EmployeeID                   7813      6414      6400      6399      6005  \
Age                     -0.758170 -0.210661 -1.743687 -1.853188 -1.196177   
DistanceFromHome        -0.887515 -0.270544 -1.010909 -0.147150  0.593216   
EnvironmentSatisfaction -1.583120  1.169861  0.252200  1.169861  0.252200   
MonthlyIncome           -0.531819 -0.081790 -0.154670 -0.645708  0.213981   
NumCompaniesWorked       0.524943 -1.078797 -0.677862 -0.677862 -1.078797   
PercentSalaryHike       -0.877232  1.582663  0.762698  1.309341  0.762698   
TotalWorkingYears       -0.549605 -0.292524 -1.320847 -1.192307 -0.549605   
YearsAtCompany          -0.327893  0.161947 -0.981014 -0.817734 -0.164613   
Atrition                 1.000000  1.000000  1.000000  1.000000  1.000000   

EmployeeID                   8366      5507      8364      7135      5506  \
Age                     -1.634185  2.088878 -1.743687 -1.196177 -0.320163   
DistanceFromHome        -1.010909 -1.010909 -1.010909 -0.640727  2.197341  

In [10]:
# Cargar el scaler previamente guardado
scaler = joblib.load('salidas/scaler.pkl')

# Seleccionar las columnas escaladas
columnas_escaladas = ['Age', 'DistanceFromHome', 'EnvironmentSatisfaction', 'MonthlyIncome', 
                      'NumCompaniesWorked', 'PercentSalaryHike','TotalWorkingYears', 'YearsAtCompany']

# Extraer las posiciones de las columnas en el scaler original
columnas_idx = [list(scaler.feature_names_in_).index(col) for col in columnas_escaladas]

# Crear un nuevo scaler solo para las columnas seleccionadas
scaler_reducido = StandardScaler()
scaler_reducido.mean_ = scaler.mean_[columnas_idx]
scaler_reducido.scale_ = scaler.scale_[columnas_idx]
scaler_reducido.var_ = scaler.var_[columnas_idx]  # Incluyendo la varianza si es necesario

# Seleccionar solo las columnas que fueron escaladas
df_pred_bajo_escalado = emp_pred_bajo[columnas_escaladas]

# Desescalar las predicciones
df_pred_bajo_original = pd.DataFrame(scaler_reducido.inverse_transform(df_pred_bajo_escalado), columns=df_pred_bajo_escalado.columns, index=df_pred_bajo_escalado.index)

# Concatenar las predicciones desescaladas con las columnas relevantes
df_pred_final = pd.concat([emp_pred_bajo[['Atrition']], df_pred_bajo_original], axis=1)

# Trasponer la tabla final
df_pred_final_traspuesta = df_pred_final.T

# Mostrar los resultados traspuestos
print(df_pred_final_traspuesta)


EmployeeID                  7813     6414     6400     6399     6005     8366  \
Atrition                     1.0      1.0      1.0      1.0      1.0      1.0   
Age                         30.0     35.0     21.0     20.0     26.0     22.0   
DistanceFromHome             2.0      7.0      1.0      8.0     14.0      1.0   
EnvironmentSatisfaction      1.0      4.0      3.0      4.0      3.0      3.0   
MonthlyIncome            40000.0  61180.0  57750.0  34640.0  75100.0  49680.0   
NumCompaniesWorked           4.0      0.0      1.0      1.0      0.0      1.0   
PercentSalaryHike           12.0     21.0     18.0     20.0     18.0     14.0   
TotalWorkingYears            7.0      9.0      1.0      2.0      7.0      1.0   
YearsAtCompany               5.0      8.0      1.0      2.0      6.0      1.0   

EmployeeID                  5507     8364     7135     5506     7459     4966  \
Atrition                     1.0      1.0      1.0      1.0      1.0      1.0   
Age                        

# Análisis y recomendaciones


***Satisfacción con el ambiente laboral:*** Hay empleados con baja satisfacción laboral (EnvironmentSatisfaction = 1 o 2). Se podría mejorar el ambiente mediante:

* Encuestas más frecuentes sobre satisfacción laboral.
* Cambios en la cultura o clima organizacional.
* Incentivos para un mejor balance vida-trabajo.

***Ingresos bajos:*** Algunos empleados tienen ingresos relativamente bajos comparados con otros en situaciones similares. Por ejemplo, el EmployeeID 6008 tiene un ingreso mensual de solo 11,290, mucho menor que el de otros con la misma antigüedad.

* Revisar si los salarios están alineados con el mercado y con el desempeño.
* Incrementar los incentivos salariales a través de bonos o ajustes anuales basados en méritos.

***Distancia desde el hogar:*** Varios empleados viven lejos del lugar de trabajo, como el EmployeeID 7459, que vive a 16 km, o el 5506, a 27 km. Considerar políticas como:

* Flexibilidad laboral, teletrabajo o esquemas híbridos.
* Subsidios de transporte o reubicación para empleados que viven lejos.

***Aumento salarial:*** Aumentos salariales bajos pueden impactar la retención, ya que algunos empleados tienen un PercentSalaryHike bajo, como el EmployeeID 7459 con solo un 11% de aumento. Revisar:

* Incrementar los aumentos salariales para mantener la competitividad.
* Implementar bonos por desempeño o promociones.

***Años en la compañia:*** Empleados con poca experiencia o antigüedad pueden sentir menos compromiso con la empresa. Esto se nota en varios empleados con menos de 2 años en la empresa o pocos años de experiencia. Estrategias para retenerlos incluyen:

* Programas de mentoría o capacitación para fomentar crecimiento interno.
* Crear planes de carrera claros para motivar a estos empleados a quedarse.

***Cantidad de compañias trabajadas***: Algunos empleados han trabajado en muchas empresas (NumCompaniesWorked alto), lo cual puede indicar falta de estabilidad o compromiso a largo plazo. Se podría ofrecer:

* Oportunidades de crecimiento interno y movilidad lateral para mantener su interés.
* Retos nuevos o proyectos más ambiciosos que aumenten su sentido de pertenencia.